<a href="https://colab.research.google.com/github/halk1311/newborn/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82_%D0%BF%D0%BE%D0%B4%D0%B1%D0%BE%D1%80%D0%B0_%D1%80%D0%B5%D1%86%D0%B5%D0%BF%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Программа:
- принимает список ингредиентов, которые есть в наличии у пользователя,
- возвращает строку с детальным списоком ингдериентв и инструкцией по приготовлению наиболее подходящего веган-рецепта

In [ ]:
! pip3 install requests #ставим модуль requests
! pip3 install beautifulsoup4 #модуль beautifulsoup

import requests as rq # работа с интернет-страницей
import re # для работы со строкой 
import pandas as pd # сохранение данных с парсинга в DataFrame
from bs4 import BeautifulSoup # парсинг нужных данных
# from IPython.display import display # для показа DF при работе в PyCharm

# Часть 1. Парсинг и создание DF

Сайт https://vege.one

Категория "Веган рецепты"

In [ ]:
# сборка списка с ссылками на рецепты
first_page = 'https://vege.one/recipes/veganskie-retsepty/?PAGEN_1='
page = rq.get(first_page)
page.encoding = 'utf8'
soup = BeautifulSoup(page.text,'html.parser')
pages = soup.find_all('a', {'class':'pagination__page ajax-trigger'})
last_page = int(pages[-2].text) # номер последней страницы
pages_list = []
add_number = 1
for link in range(1, last_page + 1):
  pages_list.append(first_page + str(add_number))
  add_number += 1
r_links_list = []
for link in pages_list:
  url = link
  page = rq.get(url)
  page.encoding = 'utf8'
  soup = BeautifulSoup(page.text,'html.parser')
  all_links = soup.find_all('a', {'class':'dds_link_title'})
  for link in all_links:
    r_links_list.append('https://vege.one' + link.get('href')) 

Функции для работы с каждым рецептом

In [ ]:
# название блюда
def get_dish_name(link):
  dish_name = link.find_all('h1', {'class':'recipe_title'})
  dish_name = dish_name[0].text
  return dish_name

# ингредиенты и граммовка
def get_ingredients(link):
  ingredient_search = link.find_all('li', {'class':'ingredient'})
  ingr_info = []
  for string in ingredient_search:
    list_with_names = string.find_all('span', {'class':'food name'})
    for elem in list_with_names:
      dish_name = elem.text
    if string.find_all('span', {'class':'measure'}) == []: #где-то не указана граммовка или кол-во ингредиента
      ingr_measure = '-'
    else:
      for elem in string.find_all('span', {'class':'measure'}):
        ingr_measure = elem.text
        ingr_measure = ingr_measure.replace('\xa0', ' ')
    ingr_info.append('%s (%s)' %(dish_name, ingr_measure))
    ingr_string = ', '.join(ingr_info)
  return ingr_string

# только название ингредиентов (далее используется для поиска)
def get_ingr_only(link):
    ingredient_search = link.find_all('li', {'class':'ingredient'})
    ingr_names = []
    for string in ingredient_search:
        list_with_names = string.find_all('span', {'class':'food name'})
        for elem in list_with_names:
            ingr_names.append(elem.text.lower())
    ingr_only_str = ', '.join(ingr_names)
    return ingr_only_str

# инструкции по приготовлению
def get_instructions(link):
  instr = link.find_all('p', {'class':'mt-0'})
  instructions = ''
  for elem in instr:
    instructions = instructions + elem.text
  instructions = re.sub('[1-9]+\n|\r', '', instructions) # чистка: удаляем ненужные символы (номера шагов, отступы)
  instructions = instructions.strip()
  instructions
  list_instr = instructions.split('.')
  if '' in list_instr:
    list_instr.remove('')
  g_list_instr = []
  for sentence in list_instr:
    sentence = sentence.strip() + '.'
    g_list_instr.append(sentence)
  str_instr = ' '.join(g_list_instr)
  return str_instr

Сборка DF

In [ ]:
recipes_dict = {}
for link in r_links_list:
  page = rq.get(link)
  page.encoding = 'utf8'
  soup = BeautifulSoup(page.text,'html.parser')
  recipes_dict[get_dish_name(soup)] = [link, get_ingredients(soup), get_ingr_only(soup), get_instructions(soup)]
recipes_pd = pd.DataFrame.from_dict(recipes_dict, orient='index', columns=["Ссылка", "Ингредиенты", "Ингредиенты(поиск)", "Инструкция"])

In [ ]:
display(recipes_pd)

,Ссылка,Ингредиенты,Ингредиенты(поиск),Инструкция
Сливочный грибной суп с рисом,https://vege.one/recipes/veganskie-retsepty/ve...,"Рис жасмин (5 ст.л), Помидор (2 шт), Шампиньон...","рис жасмин, помидор, шампиньоны, молоко кокосо...",Моем и режем на кусочки грибы. Отправляем вари...
Салат с кускусом и свёклой,https://vege.one/recipes/veganskie-retsepty/sa...,"Кускус (3 ст.л), Свёкла (1 шт), Лук зелёный (п...","кускус, свёкла, лук зелёный, огурец, соль, пер...",Моем и отвариваем свёклу до готовности. Остужа...
Маджадра,https://vege.one/recipes/veganskie-retsepty/vt...,"Чечевица зелёная (1/2 стак), Рис жасмин (1 ста...","чечевица зелёная, рис жасмин, лук репчатый, со...",Зелёную чечевицу замочить в воде комнатной тем...
Котлетки из цветной капусты и риса,https://vege.one/recipes/veganskie-retsepty/za...,"Капуста цветная (300-400 г), Рис круглозёрный ...","капуста цветная, рис круглозёрный, мука нутова...",Помыть и отварить цветную капусту до готовност...
Пророщенный маш по-корейски,https://vege.one/recipes/veganskie-retsepty/sa...,"Маш (150 г), Лук репчатый (1-2 шт), Кинза (1 п...","маш, лук репчатый, кинза, кориандр, паприка, п...","Промываем маш, заливаем кипятком и накрываем к..."
...,...,...,...,...
Роллы с огурцом и кунжутом,https://vege.one/recipes/veganskie-retsepty/za...,"Огурец (1 шт), Водоросли нори (2 листа), Рис к...","огурец, водоросли нори, рис круглозёрный, вода...",Прежде всего нам нужно правильно сварить рис. ...
Диетическое овсяное печенье с бананом,https://vege.one/recipes/veganskie-retsepty/ve...,"Овсяные хлопья (250 г), Банан (5 шт), Корица (...","овсяные хлопья, банан, корица, кардамон, муска...","Берём достаточно глубокую миску, очищаем банан..."
Аквафаба из нута,https://vege.one/recipes/veganskie-retsepty/de...,"Нут (200 г), Вода (1 л)","нут, вода",Залить сухой нут большим количеством чистой во...
Веганская рыба из нори и тофу — «Жареный карась»,https://vege.one/recipes/veganskie-retsepty/za...,"Тофу (300 г), Соль (по вкусу), Мускатный орех ...","тофу, соль, мускатный орех, мука пшеничная цел...",Разрезаем тофу на 10 ровных брусочков. Затем б...


In [ ]:
# сохраним
recipes_pd.to_csv('recipes_pd.csv', encoding="utf-16")

Всего 832 рецепта с сайта Vege.one, категория "Веганские рецепты"

Парсинг названий специй. Чтобы пользователь не вводил специи (=для удобства пользователя) по умолчанию считаем, что у него есть ВСЕ специи. + специи в блюде не так важны, как основные ингредиенты

In [ ]:
# список специй, считаем, что все специи у пользователя есть по умолчанию
first_page = 'https://vege.one/food/spetsii/?PAGEN_1='
page = rq.get(first_page)
page.encoding = 'utf8'
soup = BeautifulSoup(page.text,'html.parser')
pages = soup.find_all('a', {'class':'pagination__page ajax-trigger'})
max_page = int(pages[-2].text)
pages_list = []
add_number = 1
for link in range(1, max_page + 1):
    pages_list.append(first_page + str(add_number))
    add_number += 1
spices_list = []
for link in pages_list:
  url = link
  page = rq.get(url)
  page.encoding = 'utf8'
  soup = BeautifulSoup(page.text,'html.parser')
  all_titles = soup.find_all('h3', {'class':'title'})
  for link in all_titles:
    spices_list.append(link.text.lower())

# Часть 2. Взаимодействие с пользователем и поиск

Пользователь вводит ингредиенты

In [ ]:
ingr = input('Я помогу тебе подобрать рецепт исходя из имеющихся у тебя ингредиентов.\nРасскажи, какие ингредиенты у тебя есть. Введи их через запятую, вот так: мука рисовая, молоко кокосовое, ягоды\n  ')
user_input = ingr.lower().split(', ') 
# перевод в нижний регистр, удаление случайных пробелов, удаление повторов
# тофу, соевый соус, водоросли нори, мука пшеничная, мука рисовая, мука ржаная, кешью, арахис, молоко кокосовое, молоко овсяное, молоко миндальное, чечевица красная, чечевица зеленая, чечевица, нут, ягоды, фрукты, яблоко, помидор, огурец, перец, картофель, киноа, кунжут

Я помогу тебе подобрать рецепт исходя из имеющихся у тебя ингредиентов.
Расскажи, какие ингредиенты у тебя есть. Введи их через запятую, вот так: мука рисовая, молоко кокосовое, ягоды
  тофу, соевый соус, водоросли нори, мука пшеничная, мука рисовая, мука ржаная, кешью, арахис, молоко кокосовое, молоко овсяное, молоко миндальное, чечевица красная, чечевица зеленая, чечевица, нут, ягоды, фрукты, яблоко, помидор, огурец, перец, картофель, киноа, кунжут


Добавляем с список ингредиентов пользователя воду и растительное молоко (при условии, что пользователь добавил какое-то растительное молоко)

In [ ]:
user_ingr_list = ['вода']
for ingr in user_input:
    if re.search('молоко\D?', ingr) is not None:
        user_ingr_list.append('молоко растительное')
        user_ingr_list.append(ingr)
    else:
        user_ingr_list.append(ingr)
user_ingr_set = set(user_ingr_list)

Создаем отдельный DF, где есть индекс рецепта (из первого большого DF) и список ингредиентов, очищенный от специй

In [ ]:
# словарь где ключ - это индекс рецепта из DF, а значение - список ингредиентов (без специй)
recipes_pd = pd.read_csv('recipes_pd.csv', encoding = 'utf-16', sep =',')
# recipes_pd = pd.read_csv('/Users/alena1/Downloads/recipes_pd.csv', encoding = 'utf-16', sep =',') # при работе локально
r_ind_ingr_dict = {}
for index in recipes_pd.index:
    ingr_with_spices = recipes_pd.iloc[index]['Ингредиенты(поиск)'].split(', ')
    clean_ingr = [ingr for ingr in ingr_with_spices if ingr not in spices_list]
    r_ind_ingr_dict[index] = clean_ingr

Сравнение ингредиентов пользователя и ингредиентов в рецептах, формируем словарь, где ключ - индекс рецепта (из первого большого DF), а значение - десятичная дробь (отношение кол-ва совпадений к кол-ву ингредиентов в рецепте)

In [ ]:
ingr_match_dict = {}
for key in r_ind_ingr_dict.keys():
    ingr_set = set(r_ind_ingr_dict.get(key))
    match_score = len(user_ingr_set & ingr_set)/len(ingr_set)
    ingr_match_dict[key] = (match_score)

Делаем из словаря новый DF и сортируем, чтобы сверху были наиболее подходящие рецпты

In [ ]:
match_df = pd.DataFrame(ingr_match_dict.items(), columns=['номер', 'совпадение'])
match_df = match_df.sort_values(by='совпадение', ascending=False)

In [ ]:
match_df

,номер,совпадение
491,491,1.0
822,822,1.0
464,464,1.0
592,592,1.0
325,325,1.0
...,...,...
557,557,0.0
352,352,0.0
698,698,0.0
359,359,0.0


Взаимодействие с пользователем. Даем на выбор несколько (в данном случае 10) рецептов, если пользователю нравится название блюда, выводим состав и инструкцию по приготовлению

In [ ]:
number_of_variants = 10
for number in range(number_of_variants):
    dish_number = int(match_df.iloc[number]['номер'])
    dish_name = recipes_pd.iloc[dish_number]['Unnamed: 0']
    if number == 0:
        print(f'Магия свершилась! Лучшее совпадение - это {dish_name}')
        user_answer = input('Интересно?(да/нет)\n')
        if user_answer == 'да':
            print(f'Тогда я расскажу поподробнее.\n'
                  f'Вам понадобятся следующие ингредиенты: {recipes_pd.iloc[dish_number]["Ингредиенты"]}\n'
                  f'{recipes_pd.iloc[dish_number]["Инструкция"]}'
                  )
            break
        # elif user_answer == 'нет':
        #     continue
    else:
        print(f'Вот что еще может подойти: {dish_name}')
        user_answer = input('Интересно?(да/нет)\n')
        if user_answer.lower().strip() == 'да':
            print(f'Тогда я расскажу поподробнее.\n'
                  f'Вам понадобятся следующие ингредиенты: {recipes_pd.iloc[dish_number]["Ингредиенты"]}\n'
                  f'{recipes_pd.iloc[dish_number]["Инструкция"]}'
                  )
            break
        elif user_answer == 'нет' and number != (number_of_variants-1):
            continue
        elif user_answer == 'нет' and number == (number_of_variants-1):
            print('Упс, идеи кончились :(')

Магия свершилась! Лучшее совпадение - это Тесто пельменное
Интересно?(да/нет)
нет
Вот что еще может подойти: Ржаная закваска - видео рецепт
Интересно?(да/нет)
нет
Вот что еще может подойти: Кешью–карри
Интересно?(да/нет)
нет
Вот что еще может подойти: Яблочная пастила
Интересно?(да/нет)
нет
Вот что еще может подойти: Аюрведический чай для очищения бронхов
Интересно?(да/нет)
нет
Вот что еще может подойти: Аквафаба из нута
Интересно?(да/нет)
нет
Вот что еще может подойти: Запечённая картошка, как из костра
Интересно?(да/нет)
нет
Вот что еще может подойти: Заварной крем без глютена
Интересно?(да/нет)
нет
Вот что еще может подойти: Постные хлебцы из ржаной муки
Интересно?(да/нет)
нет
Вот что еще может подойти: Чечевичные булочки без муки
Интересно?(да/нет)
да
Тогда я расскажу поподробнее.
Вам понадобятся следующие ингредиенты: Чечевица красная (250 г), Псиллиум (2 ст.л), Вода (50 мл), Соль (1/2 ч.л), Специи (по вкусу)
Чечевицу промыть и залить большим количеством воды на ночь (это количест

Пользователь сыт и доволен! :)

# Проблемы/недочеты/areas for improvement 

1) На сайте нет единообразия в наименовании ингредиентов в оформлении карточки с ингредиентами -> трудности при сравнении ингредиентов пользователя и рецепта

Например: 
- "арахис", "арахис жареный", "арахис обжаренный"
- "молоко кокосовое", "кокосовое молоко", "кокосовое молоко фирмы Alpro"

*Решение*: усовершенствовать алгоритм сравнения

2) Пользователю предлагаются нерелевантные рецепты (напитки, супы, салаты и т.п.)

*Решение*: разбить рецепты по категориям, дать пользователю возможность выбрать категорию рецепта

3) На сайте представлены среди прочего видео-рецепты

*Решение*: исключить?